### Import Libraries

In [15]:
# data processing
import pandas as pd

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

### Load the Data...

In [2]:
# get the data
survey_df = pd.read_csv("ACME-HappinessSurvey2020.csv")

### Exploratory Data Analysis

In [4]:
# Display the data
survey_df.head()

,Y,X1,X2,X3,X4,X5,X6
0,0,3,3,3,4,2,4
1,0,3,2,3,5,4,3
2,1,5,3,3,3,3,5
3,0,5,4,3,3,3,5
4,0,5,4,3,3,3,5


In [5]:
# Display data type
survey_df.dtypes

Y     int64
X1    int64
X2    int64
X3    int64
X4    int64
X5    int64
X6    int64
dtype: object

In [8]:
# Find missing values
survey_df.isna().sum()

Y     0
X1    0
X2    0
X3    0
X4    0
X5    0
X6    0
dtype: int64

` We can already observe that there are no missing values in the dataset and that all the columns are integers. `

### A description of the features is listed below:

---

   Attributes X1 to X6 indicate the responses for each question and have values from 1 to 5 where the smaller number indicates less and the higher number indicates more towards the answer.
   
      Y = target attribute (Y) with values indicating 0 (unhappy) and 1 (happy) customers
      X1 = my order was delivered on time
      X2 = contents of my order was as I expected
      X3 = I ordered everything I wanted to order
      X4 = I paid a good price for my order
      X5 = I am satisfied with my courier
      X6 = the app makes ordering easy for me

In [9]:
# display general statistics of data
survey_df.describe()

,Y,X1,X2,X3,X4,X5,X6
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,0.547619,4.333333,2.531746,3.309524,3.746032,3.650794,4.253968
std,0.499714,0.800000,1.114892,1.023440,0.875776,1.147641,0.809311
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,4.000000,2.000000,3.000000,3.000000,3.000000,4.000000
50%,1.000000,5.000000,3.000000,3.000000,4.000000,4.000000,4.000000
75%,1.000000,5.000000,3.000000,4.000000,4.000000,4.000000,5.000000
max,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


`From the statistics above, in terms of ordered content, most of the customers expectations were sort of dashed.` This metric received the lowest average in terms of satisfaction (2.5)

In [16]:
fig, axes = plt.subplot(3,2, figsize=(10,6))
axs= sns.hisplot(survey_df, x ="X1", kde=True, color = "blue", axs = axes[0,0])

ValueError: Illegal argument(s) to subplot: (3, 2)

<Figure size 432x288 with 0 Axes>